<a href="https://colab.research.google.com/github/Mertkuludag/NDVI_Colab/blob/main/Ortalama_NDVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importlar

In [2]:
!pip install geemap


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 11.9 MB/s 
     |████████████████████████████████| 95 kB 3.3 MB/s 
     |████████████████████████████████| 187 kB 60.6 MB/s 
     |████████████████████████████████| 8.8 MB 85.7 MB/s 
     |████████████████████████████████| 1.2 MB 62.8 MB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 
     |████████████████████████████████| 46 kB 2.8 MB/s 
     |████████████████████████████████| 3.3 MB 57.7 MB/s 
     |████████████████████████████████| 3.0 MB 60.1 MB/s 
     |████████████████████████████████| 98 kB 7.9 MB/s 
     |████████████████████████████████| 1.3 MB 67.8 MB/s 
     |████████████████████████████████| 130 kB 70.1 MB/s 
     |████████████████████████████████| 128 kB 71.2 MB/s 
     |████████████████████████████████| 342 kB 65.5 MB/s 
     |████████████████████████████████| 428 kB 73.0 MB/s 
     |████████████████████████████████|

In [3]:
# Installs geemap package
import google.colab

import geemap

# Authenticates and initializes Earth Engine
import ee

ee.Authenticate()
ee.Initialize()  

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=8g8x4LUIf6qHBIkZXu9fBqNyrLseZkkfaLvufyFrsws&tc=VDKqJO6oE-h2XN8WJQQTZmFwMZNwgnIc-Ic9lzoO6uw&cc=2rfcZeETvQlJ7X3ZatpIY_KAHsEUVuPKP77I8OvNvUU

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWibOdHJ3j1umw0LA737vZEWU9b6iXheLI5K26dcRRxFUahONtpFOWw

Successfully saved authorization token.


In [3]:
# Drive Bağlantısı

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Görüntü İşleme

In [4]:
turkiye = ee.FeatureCollection("users/mkuludag93/Tr_Sinir_simp")
roi = turkiye

# Bulut Maskesi
def se2mask(image):
    quality_band = image.select('QA60')
    
    # 
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # 
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # 
    return image.updateMask(mask).divide(10000)

In [5]:
# Gece Görüntüsü ile Kent alanları maskesi

nightset = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2021-01-01","2021-11-28")


nightsetmed = nightset.median().clip(roi) 
avg_rad = nightsetmed.select('avg_rad')

def nightmask(img):
  nightexc = avg_rad.gte(7)
  img = img.updateMask(nightexc.unmask().Not())

  return(img);   

In [6]:
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
occurrence = gsw.select('occurrence')

def watermask(img):
  excludewater = occurrence.gte(5); 
  img = img.updateMask(excludewater.unmask().Not())
  return(img)

## Sentinel 2

In [7]:

# Görütüleme parametrelerini tanımlayalım
s2bands = ['B4','B3','B2']
rgbViz = {"min":0.0, "max":0.3,"bands":s2bands}

aylar = [1,2,3,4,5,6,7,8,9,10,11,12]

ndvi_liste = []

def ndviHesapla(image):
  ndvi = image.normalizedDifference(['B8', 'B4']);
  return ndvi


def sentinel2goruntu(tarih):
    se = ee.ImageCollection('COPERNICUS/S2').filterDate(
    f'2021-{tarih}-01',f'2021-{tarih}-27').filterBounds(roi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",100)).map(se2mask).map(watermask).map(ndviHesapla).mean()
    se_ndvi = se.clip(roi).rename(f'ndvi{tarih}')
    ndvi_liste.append(se_ndvi)
    return ndvi_liste

  


def band_ekle(goruntu):
  for i in range(1,len(ndvi_liste)):
    goruntu = goruntu.addBands(ndvi_liste[i])
    return goruntu


In [8]:
for ay in aylar:
  sentinel2goruntu(ay)

print(ndvi_liste)

ilk_goruntu = ndvi_liste[0]



[<ee.image.Image object at 0x7f21aad747d0>, <ee.image.Image object at 0x7f21aae5c510>, <ee.image.Image object at 0x7f21aadf8990>, <ee.image.Image object at 0x7f21aad7e390>, <ee.image.Image object at 0x7f21aad819d0>, <ee.image.Image object at 0x7f21aad81c50>, <ee.image.Image object at 0x7f21aad8c6d0>, <ee.image.Image object at 0x7f21aad99510>, <ee.image.Image object at 0x7f21aad9ac90>, <ee.image.Image object at 0x7f21aad9fb10>, <ee.image.Image object at 0x7f21aad9ff90>, <ee.image.Image object at 0x7f21aada5d90>]


In [ ]:
ndvi = band_ekle(ilk_goruntu)

## Haritada Gösterim

In [9]:
ndviParams = {min: -1, max: 1, 'palette': ['blue', 'white', 'green']}

# Haritamıza ndvi'ı
map2 = geemap.Map()
map2.centerObject(roi, 7)

map2.addLayer(ilk_goruntu, ndviParams, 'NDVI');

map2.addLayerControl()
map2


Map(center=[39.13668106908927, 35.17622716658255], controls=(WidgetControl(options=['position', 'transparent_b…

## Görüntü Exportlama

###1. Yol

In [ ]:
Adana = ee.FeatureCollection("users/mkuludag93/Sinirlar/Adana_WGS84")

In [ ]:
import os
out_dir = os.path.expanduser("~\Desktop\Goruntuler")
geemap.ee_export_image(ndvi, filename='tr_ndvi.tif', scale=10, region=Adana.geometry(), file_per_band=False)

SyntaxError: ignored

---

###2.yol

In [ ]:
# Drive Exportlama
task = ee.batch.Export.image.toDrive(**{
    'image': ndvi,
    'description': '12_Aylik_NDVI',
    'folder':'GEE',
    'scale': 100,
    'region': roi.geometry(),
    'maxPixels': 10000000000000
})
task.start()

import time 
zaman = 0
while task.active():
  print(f'Yürütülüyor (id: {task.id}), Geçen zaman {zaman}')
  zaman += time.time()
  time.sleep(5)

Yürütülüyor (id: I2GYZTHX3P75FFSNUCHH77VN), Geçen zaman 0
Yürütülüyor (id: I2GYZTHX3P75FFSNUCHH77VN), Geçen zaman 1653660041.267228
Yürütülüyor (id: I2GYZTHX3P75FFSNUCHH77VN), Geçen zaman 3307320087.6532025


KeyboardInterrupt: ignored

In [11]:
# Exporting Asset

task = ee.batch.Export.image.toAsset(**{
  'image': ndvi_liste[6],
  'description': f'12_Aylik_NDVI_{6}',
  'assetId': f'users/mkuludag93/12_Aylik_NDVI_{6}',
  'scale': 10,
  'region': roi.geometry(),
  'maxPixels': 10000000000000
  })
task.start()

